## Part (a): PyPlate Implementation

In [1]:
# Importing necessary packages and objects from PyPlate
from pyplate import Plate, Substance, Container, Recipe
import random
import pandas as pd

# Set the random seed for reproducibility
random.seed(42)

### Define Starting Substances, Catalyst, Ligands, Solvents and Containers to store them

In [2]:
# Define real molecular weights and density for Pd catalyst and ligands
Pd_catalyst = Substance.solid(name='Pd(OAc)2', mol_weight=224.50)
Pd_catalyst_container = Container("Pd_catalyst", initial_contents=[(Pd_catalyst, "10 mL")])

# Ligands
xphos = Substance.solid(name='XPhos', mol_weight=345.41)
sphos = Substance.solid(name='SPhos', mol_weight=295.35)
dppf = Substance.solid(name='dppf', mol_weight=554.63)
ligandsS = [xphos, sphos, dppf]

# Ligand Containers
xphos_container = Container("XPhos", initial_contents=[(xphos, "10 mL")])
sphos_container = Container("SPhos", initial_contents=[(sphos, "10 mL")])
dppf_container = Container("dppf", initial_contents=[(dppf, "10 mL")])
ligandsC = [xphos_container, sphos_container, dppf_container]

# Solvents
toluene = Substance.liquid(name="toluene", mol_weight=92.141, density=0.8623)
glyme = Substance.liquid(name="glyme", mol_weight=90.12, density=0.868)
TBME = Substance.liquid(name="TBME", mol_weight=88.15, density=0.74)
dichloroethane = Substance.liquid(name="dichloroethane", mol_weight=98.95, density=1.253)
solventsS = [toluene, glyme, TBME, dichloroethane]

# Solvent Containers
toluene_container = Container("toluene", initial_contents=[(toluene, "10 mL")])
glyme_container = Container("glyme", initial_contents=[(glyme, "10 mL")])
TBME_container = Container("TBME", initial_contents=[(TBME, "10 mL")])
dichloroethane_container = Container("dichloroethane", initial_contents=[(dichloroethane, "10 mL")])
solventsC = [toluene_container, glyme_container, TBME_container, dichloroethane_container]


# Generate molecular weights for Ai and Bi using a random number generator
Ai_substances = [Substance.solid(name=f"A{i}", mol_weight=round(random.uniform(10, 200),2)) for i in range(12)]
Bi_substances = [Substance.solid(name=f"B{i}", mol_weight=round(random.uniform(10, 200),2)) for i in range(12)]

Ai_containers = [Container(f"A{i}_C",  initial_contents=[(Ai_substances[i], "10 ML")])
                 for i in range(12)]
Bi_containers = [Container(f"B{i}_C",  initial_contents=[(Bi_substances[i], "10 ML")])
                 for i in range(12)]

### Initialize a 96-well plate with max volume of 500 uL per well.

In [3]:
plate = Plate('96_well_plate', max_volume_per_well='500 uL', rows=8, columns=12)

### Create a new recipe and add the components to the recipe

In [4]:
recipe = Recipe().uses(plate,Pd_catalyst_container)

for i in range(len(Ai_containers)):
    recipe.uses(Ai_containers[i])
    recipe.uses(Bi_containers[i])
    
for i in range(len(ligandsC)):
    recipe.uses(ligandsC[i])
    
for i in range(len(solventsC)):
    recipe.uses(solventsC[i])

In [5]:
## Fucntions to calculate the volumes
def calculate_volume(molar_mass_g_mol, concentration_mmol):
    # Convert molar mass from g/mol to mg/mmol (mmol/mL) and calculate volume
    concentration_mg_mmol = molar_mass_g_mol / 1000  # Convert g/mol to mg/mmol
    volume_ml = concentration_mmol / concentration_mg_mmol
    return str(round(volume_ml,3)) + " uL" 

def calculate_remaining_volume(Ai, Bi, catalyst, ligand, total):
    remain = float(total.split(" ")[0]) - (float(Ai.split(" ")[0]) + float(Bi.split(" ")[0]) +  float(catalyst.split(" ")[0]) + float(ligand.split(" ")[0]))
    return str(remain) + " uL" 

In [6]:
for row in range(plate.n_rows):
    for col in range(plate.n_columns):
        ligand_index = col // 4
        solvent_index = col // 3
        AiC = Ai_containers[col]
        BiC = Bi_containers[col]
    
        # Transfer 0.1 mmol of Ai
        Ai = Ai_substances[col]
        Ai_volume = calculate_volume(Ai.mol_weight, 0.1)
        
        # Transfer 1.1 equivalents of Bi
        Bi = Bi_substances[col]
        Bi_volume = calculate_volume(Bi.mol_weight, 0.1 * 1.1)
        
        # Transfer 10 mol% equivalents of Pd(OAc)2
        catalyst_volume = calculate_volume(Pd_catalyst.mol_weight, 0.1 * 0.10)
        
        ligandC = ligandsC[ligand_index]
        ligandS = ligandsS[ligand_index]
        # Transfer 10 mol% equivalents of ligand 
        ligand_volume = calculate_volume(ligandS.mol_weight, 0.1 * 0.15)
        
        solventC = solventsC[solvent_index]
        
        recipe.transfer(AiC, plate[row+1, col+1], Ai_volume)
        recipe.transfer(BiC, plate[row+1, col+1], Bi_volume)        
        recipe.transfer(Pd_catalyst_container, plate[row+1, col+1], catalyst_volume)
        recipe.transfer(ligandC, plate[row+1, col+1], ligand_volume)
        
        # Fill the remaining volume of each well with the solvent
        remaining_volume = calculate_remaining_volume(Ai_volume, Bi_volume, catalyst_volume, ligand_volume, "200 uL")
        recipe.transfer(solventC, plate[row+1, col+1], remaining_volume)

results = recipe.bake()
plate = results[plate.name]
# recipe.visualize(what=plate, mode='final', unit='uL', timeframe=0)

#### Adding Substance A1 to Well A2

In [7]:
recipe.visualize(what=plate, mode='final', unit='uL', timeframe=5)

C:\Users\meesa\anaconda3\envs\graph_ds_env\Lib\site-packages\pyplate\pyplate.py:2215: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = data.applymap(numpy.vectorize(helper, cache=True, otypes='d'))


,1,2,3,4,5,6,7,8,9,10,11,12
A,200.0,6.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
D,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Adding Substance B1 to Well A2

In [8]:
recipe.visualize(what=plate, mode='final', unit='uL', timeframe=6)

C:\Users\meesa\anaconda3\envs\graph_ds_env\Lib\site-packages\pyplate\pyplate.py:2215: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = data.applymap(numpy.vectorize(helper, cache=True, otypes='d'))


,1,2,3,4,5,6,7,8,9,10,11,12
A,200.0,9.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
D,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Adding Catalyst to Well A2

In [9]:
recipe.visualize(what=plate, mode='final', unit='uL', timeframe=7)

,1,2,3,4,5,6,7,8,9,10,11,12
A,200.0,9.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
D,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Adding Ligand to Well A2

In [10]:
recipe.visualize(what=plate, mode='final', unit='uL', timeframe=8)

,1,2,3,4,5,6,7,8,9,10,11,12
A,200.0,9.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
D,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Filling the Well A2 upto 200uL with solvent

In [11]:
recipe.visualize(what=plate, mode='final', unit='uL', timeframe=9)

,1,2,3,4,5,6,7,8,9,10,11,12
A,200.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
D,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Final Step

In [12]:
recipe.visualize(what=plate, mode='final', unit='uL', timeframe='all')

,1,2,3,4,5,6,7,8,9,10,11,12
A,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0
B,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0
C,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0
D,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0
E,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0
F,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0
G,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0
H,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0


### Cross Coupling Reactions Matrix Visualization

In [13]:
ligands = ['XPhos', 'SPhos', 'dppf']
solvents = ['Toluene', 'Glyme', 'TBME', 'Dichloroethane']
# Generate the heatmap data
data = []
for row in range(plate.n_rows):
    row_data = []
    for col in range(plate.n_columns):
        ligand_index = col // 4
        solvent_index = col // 3
        row_data.append(f"A{col+1}"+" "+f"B{col+1}"+" "+"Pd_Cat"+" "+ligands[ligand_index]+" "+solvents[solvent_index])
    data.append(row_data)
    
plate_design =  pd.DataFrame(data)
plate_design

,0,1,2,3,4,5,6,7,8,9,10,11
0,A1 B1 Pd_Cat XPhos Toluene,A2 B2 Pd_Cat XPhos Toluene,A3 B3 Pd_Cat XPhos Toluene,A4 B4 Pd_Cat XPhos Glyme,A5 B5 Pd_Cat SPhos Glyme,A6 B6 Pd_Cat SPhos Glyme,A7 B7 Pd_Cat SPhos TBME,A8 B8 Pd_Cat SPhos TBME,A9 B9 Pd_Cat dppf TBME,A10 B10 Pd_Cat dppf Dichloroethane,A11 B11 Pd_Cat dppf Dichloroethane,A12 B12 Pd_Cat dppf Dichloroethane
1,A1 B1 Pd_Cat XPhos Toluene,A2 B2 Pd_Cat XPhos Toluene,A3 B3 Pd_Cat XPhos Toluene,A4 B4 Pd_Cat XPhos Glyme,A5 B5 Pd_Cat SPhos Glyme,A6 B6 Pd_Cat SPhos Glyme,A7 B7 Pd_Cat SPhos TBME,A8 B8 Pd_Cat SPhos TBME,A9 B9 Pd_Cat dppf TBME,A10 B10 Pd_Cat dppf Dichloroethane,A11 B11 Pd_Cat dppf Dichloroethane,A12 B12 Pd_Cat dppf Dichloroethane
2,A1 B1 Pd_Cat XPhos Toluene,A2 B2 Pd_Cat XPhos Toluene,A3 B3 Pd_Cat XPhos Toluene,A4 B4 Pd_Cat XPhos Glyme,A5 B5 Pd_Cat SPhos Glyme,A6 B6 Pd_Cat SPhos Glyme,A7 B7 Pd_Cat SPhos TBME,A8 B8 Pd_Cat SPhos TBME,A9 B9 Pd_Cat dppf TBME,A10 B10 Pd_Cat dppf Dichloroethane,A11 B11 Pd_Cat dppf Dichloroethane,A12 B12 Pd_Cat dppf Dichloroethane
3,A1 B1 Pd_Cat XPhos Toluene,A2 B2 Pd_Cat XPhos Toluene,A3 B3 Pd_Cat XPhos Toluene,A4 B4 Pd_Cat XPhos Glyme,A5 B5 Pd_Cat SPhos Glyme,A6 B6 Pd_Cat SPhos Glyme,A7 B7 Pd_Cat SPhos TBME,A8 B8 Pd_Cat SPhos TBME,A9 B9 Pd_Cat dppf TBME,A10 B10 Pd_Cat dppf Dichloroethane,A11 B11 Pd_Cat dppf Dichloroethane,A12 B12 Pd_Cat dppf Dichloroethane
4,A1 B1 Pd_Cat XPhos Toluene,A2 B2 Pd_Cat XPhos Toluene,A3 B3 Pd_Cat XPhos Toluene,A4 B4 Pd_Cat XPhos Glyme,A5 B5 Pd_Cat SPhos Glyme,A6 B6 Pd_Cat SPhos Glyme,A7 B7 Pd_Cat SPhos TBME,A8 B8 Pd_Cat SPhos TBME,A9 B9 Pd_Cat dppf TBME,A10 B10 Pd_Cat dppf Dichloroethane,A11 B11 Pd_Cat dppf Dichloroethane,A12 B12 Pd_Cat dppf Dichloroethane
5,A1 B1 Pd_Cat XPhos Toluene,A2 B2 Pd_Cat XPhos Toluene,A3 B3 Pd_Cat XPhos Toluene,A4 B4 Pd_Cat XPhos Glyme,A5 B5 Pd_Cat SPhos Glyme,A6 B6 Pd_Cat SPhos Glyme,A7 B7 Pd_Cat SPhos TBME,A8 B8 Pd_Cat SPhos TBME,A9 B9 Pd_Cat dppf TBME,A10 B10 Pd_Cat dppf Dichloroethane,A11 B11 Pd_Cat dppf Dichloroethane,A12 B12 Pd_Cat dppf Dichloroethane
6,A1 B1 Pd_Cat XPhos Toluene,A2 B2 Pd_Cat XPhos Toluene,A3 B3 Pd_Cat XPhos Toluene,A4 B4 Pd_Cat XPhos Glyme,A5 B5 Pd_Cat SPhos Glyme,A6 B6 Pd_Cat SPhos Glyme,A7 B7 Pd_Cat SPhos TBME,A8 B8 Pd_Cat SPhos TBME,A9 B9 Pd_Cat dppf TBME,A10 B10 Pd_Cat dppf Dichloroethane,A11 B11 Pd_Cat dppf Dichloroethane,A12 B12 Pd_Cat dppf Dichloroethane
7,A1 B1 Pd_Cat XPhos Toluene,A2 B2 Pd_Cat XPhos Toluene,A3 B3 Pd_Cat XPhos Toluene,A4 B4 Pd_Cat XPhos Glyme,A5 B5 Pd_Cat SPhos Glyme,A6 B6 Pd_Cat SPhos Glyme,A7 B7 Pd_Cat SPhos TBME,A8 B8 Pd_Cat SPhos TBME,A9 B9 Pd_Cat dppf TBME,A10 B10 Pd_Cat dppf Dichloroethane,A11 B11 Pd_Cat dppf Dichloroethane,A12 B12 Pd_Cat dppf Dichloroethane


### Part (a) Explaination

Here's a brief documentation for the provided code:

### Import Statements:
        Plate, Substance, Container, and Recipe are imported from PyPlate.
        The random module is imported for generating random numbers.

### Random Seed:
        The random seed is set to 42 using random.seed(42) for reproducibility of random number generation.

### Defining Substances and Containers:
- <b>Ai_substances and Bi_substances</b>: Lists of solid substances (A0 to A11 and B0 to B11 respectively) generated using random molecular weights between 10 and 200. Each substance is stored in its corresponding container with an initial volume of 10 mL.
                
- <b>Pd_catalyst</b>: A solid substance Pd(OAc)2 with a molecular weight of 224.50. It is stored in a container named "Pd_catalyst" with an initial volume of 10 mL.

- <b>Ligands (xphos, sphos, dppf)</b>: Solid substances representing different ligands with specific molecular weights. These ligands are stored in separate containers (xphos_container, sphos_container, dppf_container) each with an initial volume of 10 mL.

- <b>Solvents (toluene, glyme, TBME, dichloroethane)</b>: Liquid substances representing different solvents with molecular weights and densities. Each solvent is stored in its respective container with an initial volume of 10 mL.


### Summary:
- The code starts by initializing starting materials A{i} and B{i}, where i ranges from 0 to 11, catalysts, ligands, solvents and stores them in container with volume of 10 mL.

- A plate with 96 wells in 8 rows and 12 columns is created with maximum volume of 500 uL per well.
- A recipe is created and info of all the materials that will be used in the experimentation is added to it.
- In a loop, starting materials A, B and catalyst, a ligand and a solvent from the available list is added to each well.
- The recipe has a total of 480 steps and each well contains 5 steps.
    - Transfering one of the A from its container to the well.
    - Transfering one of the B from its container to the well.
    - Transfering the Catalyst from its container to the well.
    - Transfering one of the Ligand from its container to the well.
    - Transfering one of the Solvent  from its container to the well.
- recipe.bake executes the steps.

## Part (b): PyPlate API Modification Proposal


To introduce the concept of tags and relative quantities like "1.1 * A" in PyPlate, we would need to make the following changes:

1. **Substance Class**: Modify the `Substance` class to include a property for tags, which would be a list of strings.

2. **Transfer Method**: Enhance the transfer method in the `Recipe` class to accept expressions involving tags, parse them, and calculate the relative quantities.

3. **Validation**: Implement a validation method to ensure physical reasonableness, such as checking that the final volume does not exceed the well capacity or that the sum of equivalents does not exceed stoichiometric constraints.

4. **Documentation**: Update docstrings in the `Substance` and `Recipe` classes to reflect the new tagging feature and usage.

Here's an example of how the docstrings might be updated to explain the new API behavior:

### Substance Class

##### Current docstring

``` markdown
class Substance:
    
    """
    An abstract chemical or biological entity (e.g., reagent, enzyme, solvent, etc.). Immutable.
    Enzymes are assumed to require zero volume.

    Attributes:
        name: Name of substance.
        mol_weight: Molecular weight (g/mol).
        specific_activity: Activity units per mass if `Substance` is an enzyme (U/g).
        density: Density if `Substance` is a liquid (g/mL).
        concentration: Calculated concentration if `Substance` is a liquid (mol/mL).
        molecule: `cctk.Molecule` if provided.
    """
```

##### Updated docstring
class Substance:

    """
    An abstract chemical or biological entity (e.g., reagent, enzyme, solvent, etc.). Immutable.
    Enzymes are assumed to require zero volume.

    Attributes:
        name: Name of substance.
        mol_weight: Molecular weight (g/mol).
        specific_activity: Activity units per mass if `Substance` is an enzyme (U/g).
        density: Density if `Substance` is a liquid (g/mL).
        tags: String labels associated with the substance to be used in specifying relative quantities.
        concentration: Calculated concentration if `Substance` is a liquid (mol/mL).
        molecule: `cctk.Molecule` if provided.
    """


### Recipe Class - Transfer

##### Current docstring
      def transfer(self, source: Container | Plate | PlateSlicer, destination: Container | Plate | PlateSlicer,
                 quantity: str) -> None:
        """
        Adds a step to the recipe which will move quantity from source to destination.
        Note that all Substances in the source will be transferred in proportion to their respective ratios.
        quantity represents the amount of substance to transfer from source to destination in terms of volume (Ex: uL, mL etc)

        """

##### Updated docstring
      def transfer(self, source: Container | Plate | PlateSlicer, destination: Container | Plate | PlateSlicer,
                 quantity: str) -> None:
        """
        Adds a step to the recipe which will move quantity from source to destination.
        Note that all Substances in the source will be transferred in proportion to their respective ratios.
        quantity can represent the following
            - amount of substance to transfer from source to destination in terms of volume (Ex: uL, mL etc)
            - relative quantity of a tagged substance to transfer from source to destination.
        """
#### Implementation details for Transfer function
- The `transfer` method now accepts expressions involving tags and calculate the relative quantities.
- Step 1: Check if the quantity is a string and involves a tag.
    - Use isinstance(quantity, str) and * in quatity as a check.
- Step 2: Parse the quatity using text split or regular expersions to extract the Tag and the multiplier.
- Step 3: Search the recipe for the extracted Tag.
    - If the Tag is found, extract the volume details of the Tagged substance.
- Step 4: Using the multiplier and the volume of the Tagged substance, calculate the amount of current substance to be transferred.


Example:
```python
# Associate the substance with a tag
A1.tag('A')

# Use the tag in a transfer to specify a relative quantity
recipe.transfer(source=B1, destination=well, quantity="1.1 * A")
```

This would transfer an amount of B1 equivalent to 1.1 times the amount associated with the tag 'A'.

#### Validation Method:

##### validate_quantities(...)
Ensures that all quantities specified in the recipe are physically reasonable. Checks include verifying that well volumes are not exceeded and that relative amounts do not result in impossible scenarios.

### Ensuring Physical Reasonableness

To ensure that quantities are physically reasonable, implement checks such as:

- Total volume in a well should not exceed its maximum capacity.
- For liquid transfers, ensure that the density of the substance is taken into account to avoid overfilling.

Unit tests could be written to validate the correctness of these checks under various scenarios, ensuring robustness and reliability of the recipe execution.